# Imports

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
import matplotlib.pyplot as plt
from pathlib import Path

Using TensorFlow backend.


## Import Data

In [2]:
data_train = pd.read_csv('../data/train_rounds_filtered_merged.csv', header=None)
data_test = pd.read_csv('../data/test_rounds_filtered_merged.csv', header=None)

cards = [
# Diamonds
'DA','DK','DQ','DJ','D10','D9','D8','D7','D6',
# Hearts
'HA','HK','HQ','HJ','H10','H9','H8','H7','H6',
# Spades
'SA','SK','SQ','SJ','S10','S9','S8','S7','S6',
# Clubs
'CA','CK','CQ','CJ','C10','C9','C8','C7','C6'
]

trump = ['trump']

In [3]:
# data_train
data_train.columns = cards + trump
data_train.head()

,DA,DK,DQ,DJ,D10,D9,D8,D7,D6,HA,...,CA,CK,CQ,CJ,C10,C9,C8,C7,C6,trump
0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,1,0,4
1,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0
2,0,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,3
3,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,1,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [4]:
# data_test
data_test.columns = cards + trump
data_test.head()

,DA,DK,DQ,DJ,D10,D9,D8,D7,D6,HA,...,CA,CK,CQ,CJ,C10,C9,C8,C7,C6,trump
0,0,1,0,0,1,0,1,1,1,0,...,0,0,0,0,1,0,0,0,0,5
1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,5
2,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
3,0,1,0,0,0,1,1,0,0,0,...,1,1,0,0,0,1,0,0,1,3
4,1,0,0,1,0,0,0,0,1,1,...,0,0,0,1,0,0,0,1,0,2


add data column with NrD, NrH, NrS, NrC, NrA, Nr6

In [5]:
# data_train
data_train.insert(loc=len(data_train.columns), column='NrD', value=data_train.iloc[:,0:9].sum(axis=1) / 9.0)
data_train.insert(loc=len(data_train.columns), column='NrH', value=data_train.iloc[:,9:18].sum(axis=1) / 9.0)
data_train.insert(loc=len(data_train.columns), column='NrS', value=data_train.iloc[:,18:27].sum(axis=1) / 9.0)
data_train.insert(loc=len(data_train.columns), column='NrC', value=data_train.iloc[:,27:36].sum(axis=1) / 9.0)

data_train.insert(loc=len(data_train.columns), column='NrA', value=data_train.iloc[:,0:36:9].sum(axis=1) / 4.0)
data_train.insert(loc=len(data_train.columns), column='Nr6', value=data_train.iloc[:,8:36:9].sum(axis=1) / 4.0)
data_train.head()

,DA,DK,DQ,DJ,D10,D9,D8,D7,D6,HA,...,C8,C7,C6,trump,NrD,NrH,NrS,NrC,NrA,Nr6
0,0,0,0,0,0,0,1,0,0,0,...,1,1,0,4,0.111111,0.333333,0.222222,0.333333,0.50,0.25
1,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0.222222,0.222222,0.222222,0.333333,0.00,0.00
2,0,0,0,1,0,1,0,0,0,0,...,0,0,1,3,0.222222,0.333333,0.111111,0.333333,0.00,0.25
3,0,1,1,0,0,1,0,0,0,0,...,1,0,0,1,0.333333,0.333333,0.222222,0.111111,0.25,0.00
4,1,0,1,0,1,1,0,0,0,0,...,0,0,0,6,0.444444,0.333333,0.222222,0.000000,0.25,0.00


In [6]:
# data_test
data_test.insert(loc=len(data_test.columns), column='NrD', value=data_test.iloc[:,0:9].sum(axis=1) / 9.0)
data_test.insert(loc=len(data_test.columns), column='NrH', value=data_test.iloc[:,9:18].sum(axis=1) / 9.0)
data_test.insert(loc=len(data_test.columns), column='NrS', value=data_test.iloc[:,18:27].sum(axis=1) / 9.0)
data_test.insert(loc=len(data_test.columns), column='NrC', value=data_test.iloc[:,27:36].sum(axis=1) / 9.0)

data_test.insert(loc=len(data_test.columns), column='NrA', value=data_test.iloc[:,0:36:9].sum(axis=1) / 4.0)
data_test.insert(loc=len(data_test.columns), column='Nr6', value=data_test.iloc[:,8:36:9].sum(axis=1) / 4.0)
data_test.head()

,DA,DK,DQ,DJ,D10,D9,D8,D7,D6,HA,...,C8,C7,C6,trump,NrD,NrH,NrS,NrC,NrA,Nr6
0,0,1,0,0,1,0,1,1,1,0,...,0,0,0,5,0.555556,0.111111,0.222222,0.111111,0.00,0.25
1,0,0,0,1,0,0,0,1,0,0,...,0,1,0,5,0.222222,0.222222,0.444444,0.111111,0.00,0.50
2,0,1,1,1,0,0,0,0,0,0,...,1,1,0,0,0.333333,0.222222,0.111111,0.333333,0.00,0.25
3,0,1,0,0,0,1,1,0,0,0,...,0,0,1,3,0.333333,0.111111,0.111111,0.444444,0.25,0.25
4,1,0,0,1,0,0,0,0,1,1,...,0,1,0,2,0.333333,0.111111,0.333333,0.222222,0.50,0.25


In [7]:
# x_train = data_train.filter(items=['DJ', 'D9', 'HJ', 'H9', 'SJ', 'S9', 'CJ', 'C9', 'NrD', 'NrH', 'NrS', 'NrC', 'NrA', 'Nr6']) 
x_train = data_train.drop('trump', axis='columns', inplace=False)
# x_train = data_train
x_train.head()

,DA,DK,DQ,DJ,D10,D9,D8,D7,D6,HA,...,C9,C8,C7,C6,NrD,NrH,NrS,NrC,NrA,Nr6
0,0,0,0,0,0,0,1,0,0,0,...,0,1,1,0,0.111111,0.333333,0.222222,0.333333,0.50,0.25
1,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0.222222,0.222222,0.222222,0.333333,0.00,0.00
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0.222222,0.333333,0.111111,0.333333,0.00,0.25
3,0,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0.333333,0.333333,0.222222,0.111111,0.25,0.00
4,1,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0.444444,0.333333,0.222222,0.000000,0.25,0.00


In [8]:
# x_test = data_test.filter(items=['DJ', 'D9', 'HJ', 'H9', 'SJ', 'S9', 'CJ', 'C9', 'NrD', 'NrH', 'NrS', 'NrC', 'NrA', 'Nr6']) 
x_test = data_test.drop('trump', axis='columns', inplace=False)
# x_test = data_test
x_test.head()

,DA,DK,DQ,DJ,D10,D9,D8,D7,D6,HA,...,C9,C8,C7,C6,NrD,NrH,NrS,NrC,NrA,Nr6
0,0,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0.555556,0.111111,0.222222,0.111111,0.00,0.25
1,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0.222222,0.222222,0.444444,0.111111,0.00,0.50
2,0,1,1,1,0,0,0,0,0,0,...,1,1,1,0,0.333333,0.222222,0.111111,0.333333,0.00,0.25
3,0,1,0,0,0,1,1,0,0,0,...,1,0,0,1,0.333333,0.111111,0.111111,0.444444,0.25,0.25
4,1,0,0,1,0,0,0,0,1,1,...,0,0,1,0,0.333333,0.111111,0.333333,0.222222,0.50,0.25


In [ ]:
y_train_label = data_train['trump']
y_train = keras.utils.to_categorical(y_train_label, num_classes=7)

model = keras.Sequential()
model.add(keras.layers.Dense(42, activation='relu', input_shape=[42]))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(42, activation='relu'))
model.add(keras.layers.Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=100, batch_size=2500, shuffle=True)

Epoch 1/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.7995 - accuracy: 0.3803
Epoch 2/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.7638 - accuracy: 0.3829
Epoch 3/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.7615 - accuracy: 0.3829
Epoch 4/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.7548 - accuracy: 0.3829
Epoch 5/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.7231 - accuracy: 0.3829
Epoch 6/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.6300 - accuracy: 0.3883
Epoch 7/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.5428 - accuracy: 0.4307
Epoch 8/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.4035 - accuracy: 0.4792
Epoch 9/100
1650676/1650676 [==============================] - 5s 3us/step - loss: 1.2373 - accuracy: 0.5324
Epoch 10/100
165067

In [ ]:
model.summary()
print(model.get_weights())

In [ ]:
plt.plot(history.history['loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['Train', 'Val'], loc='upper left')

In [ ]:
history.history['accuracy'][-1]

# Predict

In [ ]:
y_pred = model.predict(x_test)
print(y_pred)

In [ ]:
print(y_pred[0])
result = np.where(y_pred[0] == np.amax(y_pred[0]))
print(x_test.head(1))
print(result[0][0])

In [ ]:
TOTAL_COUNT = len(x_test)
CORRECT_COUNT = 0

for i in range(0,len(y_pred)):
    global CORRECT_TRUMP
    
    predicted_trump = np.where(y_pred[i] == np.amax(y_pred[i]))[0][0]
    real_trump = data_test.head(i+1).iloc[0]['trump']
    
    print(predicted_trump)
    print(real_trump)
    
    if int(predicted_trump) == int(real_trump):
        CORRECT_COUNT = CORRECT_COUNT + 1

accuracy = (CORRECT_COUNT / TOTAL_COUNT) * 100
print(f'Total Count  : {TOTAL_COUNT}')
print(f'Correct Count: {CORRECT_COUNT}')
print(f'Accuracy     : {accuracy}')

### Versions

| Version | CSV | epoch | Batch size | Layers                    | Loss | Acc |
| ------- | --- | ----- | ---------- | ------------------------- | ---- | --- |
| V0      | 1   | 100   | 2500       | 9 relu & 1 softmax        | -    | -   |
